# 用户和活动关联关系处理


整个数据集中活动数目（events.csv）太多，所以下面的处理我们找出只在训练集和测试集中出现的活动和用户集合，并对他们重新编制索引

In [1]:
#保存数据
import pickle

import itertools

#处理事件字符串
import datetime

import numpy as np
import scipy.io as sio
import scipy.sparse as ss

#相似度/距离
import scipy.spatial.distance as ssd

from collections import defaultdict
from sklearn.preprocessing import normalize

In [2]:
 """
我们只关心train和test中出现的user和event，因此重点处理这部分关联数据

train.csv 有6列：
user：用户ID
event：活动ID
invited：是否被邀请（0/1）
timestamp：ISO-8601 UTC格式时间字符串，表示用户看到该活动的时间
interested, and not_interested

Test.csv 除了没有interested, and not_interested，其余列与train相同
 """
    
# 统计训练集中有多少不同的用户的events
uniqueUsers = set()
uniqueEvents = set()

#倒排表
#统计每个用户参加的活动   / 每个活动参加的用户
eventsForUser = defaultdict(set)
usersForEvent = defaultdict(set)
    
for filename in ["train.csv", "test.csv"]:
    f = open(filename, 'r')
    
    #忽略第一行（列名字）
    f.readline().strip().split(",")
    
    for line in f:    #对每条记录
        cols = line.strip().split(",")
        uniqueUsers.add(cols[0])   #第一列为用户ID
        uniqueEvents.add(cols[1])   #第二列为活动ID
        
        #eventsForUser[cols[0]].add(cols[1])    #该用户参加了这个活动
        #usersForEvent[cols[1]].add(cols[0])    #该活动被用户参加
    f.close()


n_uniqueUsers = len(uniqueUsers)
n_uniqueEvents = len(uniqueEvents)

print("number of uniqueUsers :%d" % n_uniqueUsers)
print("number of uniqueEvents :%d" % n_uniqueEvents)

#用户关系矩阵表，可用于后续LFM/SVD++处理的输入
#这是一个稀疏矩阵，记录用户对活动感兴趣
userEventScores = ss.dok_matrix((n_uniqueUsers, n_uniqueEvents))
userIndex = dict()
eventIndex = dict()

#重新编码用户索引字典
for i, u in enumerate(uniqueUsers):
    userIndex[u] = i

#重新编码活动索引字典    
for i, e in enumerate(uniqueEvents):
    eventIndex[e] = i

n_records = 0
ftrain = open("train.csv", 'r')
ftrain.readline()
for line in ftrain:
    cols = line.strip().split(",")
    i = userIndex[cols[0]]  #用户
    j = eventIndex[cols[1]] #活动
    #print("i = %d ---j = %d" % (i ,j))
    eventsForUser[i].add(j)    #该用户参加了这个活动
    usersForEvent[j].add(i)    #该活动被用户参加
        
    #userEventScores[i, j] = int(cols[4]) - int(cols[5])   #interested - not_interested
    score = int(cols[4])
    #if score == 0:  #0在稀疏矩阵中表示该元素不存在，因此借用-1表示interested=0
    #userEventScores[i, j] = -1
    #else:
    userEventScores[i, j] = score
#print("**** eventsForUser  ***")
#print(eventsForUser)
ftrain.close()

  
##统计每个用户参加的活动，后续用于将用户朋友参加的活动影响到用户
pickle.dump(eventsForUser, open("PE_eventsForUser.pkl", 'wb'))
##统计活动参加的用户
pickle.dump(usersForEvent, open("PE_usersForEvent.pkl", 'wb'))

#保存用户-活动关系矩阵R，以备后用
sio.mmwrite("PE_userEventScores", userEventScores)


#保存用户索引表
pickle.dump(userIndex, open("PE_userIndex.pkl", 'wb'))
#保存活动索引表
pickle.dump(eventIndex, open("PE_eventIndex.pkl", 'wb'))

    
# 为了防止不必要的计算，我们找出来所有关联的用户 或者 关联的event
# 所谓的关联用户，指的是至少在同一个event上有行为的用户pair
# 关联的event指的是至少同一个user有行为的event pair
uniqueUserPairs = set()
uniqueEventPairs = set()
for event in uniqueEvents:
    i = eventIndex[event]
    #print(i)
    #print("*****************************")
    users = usersForEvent[i]    #每个users都是set
    #print(users)    
    if len(users) > 2:
        uniqueUserPairs.update(itertools.combinations(users, 2))

for user in uniqueUsers:
    u = userIndex[user]
    events = eventsForUser[u]
    if len(events) > 2:
        uniqueEventPairs.update(itertools.combinations(events, 2))
 
#保存用户-事件关系对索引表
pickle.dump(uniqueUserPairs, open("FE_uniqueUserPairs.pkl", 'wb'))
pickle.dump(uniqueEventPairs, open("PE_uniqueEventPairs.pkl", 'wb'))

number of uniqueUsers :3391
number of uniqueEvents :13418


In [3]:
#训练集和测试集中出现的用户数目和事件数目远小于users.csv出现的用户数和events.csv出现的事件数

In [4]:
print(uniqueUserPairs)

{(780, 725), (1061, 676), (1580, 340), (3114, 2347), (2176, 1205), (1596, 831), (591, 2841), (2713, 1490), (1637, 1905), (280, 1723), (1679, 672), (1562, 3202), (1591, 116), (1797, 2361), (3082, 1204), (2708, 1235), (164, 789), (1086, 1636), (1008, 537), (3102, 999), (1450, 2812), (2166, 789), (115, 1685), (2097, 306), (3128, 1994), (685, 1205), (763, 1793), (2247, 3328), (2597, 229), (1081, 356), (1632, 250), (2187, 432), (1639, 1773), (1985, 1998), (1586, 1531), (1913, 647), (3372, 2960), (1078, 331), (3332, 1023), (2725, 2173), (613, 3272), (1087, 235), (2732, 2895), (204, 244), (633, 1502), (10, 1549), (912, 765), (3281, 1406), (1082, 2025), (3191, 1303), (1465, 999), (3343, 1138), (1716, 360), (3361, 1930), (139, 780), (3080, 3354), (2669, 1333), (252, 2931), (1701, 3332), (1073, 2963), (720, 1477), (148, 1078), (1632, 2203), (736, 832), (1861, 1510), (613, 626), (763, 333), (323, 3272), (2708, 426), (1563, 612), (2053, 1563), (39, 3260), (2696, 1222), (3083, 1490), (2989, 319), (